In [5]:
import os
import pandas as pd
import time
import numpy as np
from matplotlib import pyplot as plt
import itertools
import numpy as np
from scipy import stats
import pylab as pl
from sklearn import svm, linear_model
from sklearn.model_selection import cross_validate
from collections import OrderedDict
from scipy import linalg
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV

In [2]:
#Read the pickel objects

path_to_save="D:\\Courses\\InfoStorage\\Project\\Data\\Numpy_Objects\\AllFeaturesWithAllWordEmbeddings\\"

#X = pd.read_pickle(path_to_save+"AllFeaturesCombined.pkl")
#Train = pd.read_pickle(path_to_save+"Train.pkl")
Test= pd.read_pickle(path_to_save+"Test.pkl")
train = np.loadtxt(path_to_save+'allFeaturesWithWord2VecEmbeddingsall_train.txt')
Xp = np.loadtxt(path_to_save+'allFeaturesWithWord2VecEmbeddingsAfterPairWiseall_Xp.txt')
yp = np.loadtxt(path_to_save+'allFeaturesWithWord2VecEmbeddingsAfterPairWiseall_yp.txt')
test = np.loadtxt(path_to_save+'allFeaturesWithWord2VecEmbeddingsall_test.txt')
query_item_order = np.loadtxt(path_to_save+'allFeaturesWithWord2VecEmbeddingsall_query_item_order.txt')
labels = np.loadtxt(path_to_save+'allFeaturesWithWord2VecEmbeddingsall_labels.txt')
query_item_order_test = np.loadtxt(path_to_save+'allFeaturesWithWord2VecEmbeddingsall_query_item_order_test.txt')
#labels_test = np.loadtxt(path_to_save+'allFeaturesWithWord2VecEmbeddingsall_labels_test.txt')

In [10]:
def computeSVM(x_train,y_train):
    param_grid = [{'C': [0.1,1, 10, 100, 1000]}]
    clf_grid = GridSearchCV(svm.SVC(kernel='linear'),param_grid,cv= 5)
    clf_grid.fit(x_train, y_train)
    print(clf_grid.best_params_)
    return clf_grid

In [11]:
clfb = computeSVM(Xp,yp)

KeyboardInterrupt: 

# Best

In [8]:
# Fit the Model
#clfb = svm.SVC(kernel='rbf', C=.1,gamma=0.001)
#clfb.fit(Xp, yp)
coef = clfb.coef_.ravel() / linalg.norm(clfb.coef_)
#perform the prediction 
pred= np.dot(test, coef)
# Process before Writing to a file
print(pred.shape)
print(len(pred))
odBest=Test['queryId'].value_counts(sort=True, ascending=True)
odBest = dict(odBest)
odBest= OrderedDict(sorted(odBest.items()))

AttributeError: coef_ is only available when using a linear kernel

In [ ]:
#Calculate the Scores and Get the order

q_i_b={}

s=0
e=0
for key in odBest:
    s=e
    e=s+odBest.get(key)
    print("Start:%d, End %d" %(s,e))
    subset= query_item_order_test[s:e]
    query_item_order=[]
    query_item_score=[]
    for item in subset:
        query_item_order.append(item[1])
        query_item_score.append(pred[s])
        s=s+1
    temp=np.argsort(query_item_score)[::-1]
    temp=temp.tolist()
    query_item_order= np.array(query_item_order)
    query_item_order = query_item_order[temp]
    q_i_b[key]= query_item_order.tolist()

In [ ]:
# Write to a file 
f= open("D:\\Courses\\InfoStorage\\Project\\Data\\submissions\\RankSVMAllFeaturesWithAllEmbeddingsBest.txt","w")
for key in q_i_b:
    f.write("%s " % key)
    for index,item in enumerate(q_i_b.get(key)):
        ll=len(q_i_b.get(key))-1
        item=int(item)
        if index < ll:
            f.write("%s," % item)
        else:
            f.write("%s" % item)
    f.write("\n")
f.close()

# Basic

In [ ]:
# Fit the Model
clf = svm.SVC(kernel='linear', C=.1)
clf.fit(Xp, yp)
coef = clf.coef_.ravel() / linalg.norm(clf.coef_)

In [ ]:
#perform the prediction 
pred= np.dot(test, coef)

In [ ]:
# Process before Writing to a file
print(pred.shape)
print(len(pred))
od=Test['queryId'].value_counts(sort=True, ascending=True)
od = dict(od)
od= OrderedDict(sorted(od.items()))

In [ ]:
#Calculate the Scores and Get the order

q_i={}

s=0
e=0
for key in od:
    s=e
    e=s+od.get(key)
    print("Start:%d, End %d" %(s,e))
    subset= query_item_order_test[s:e]
    query_item_order=[]
    query_item_score=[]
    for item in subset:
        query_item_order.append(item[1])
        query_item_score.append(pred[s])
        s=s+1
    temp=np.argsort(query_item_score)[::-1]
    temp=temp.tolist()
    query_item_order= np.array(query_item_order)
    query_item_order = query_item_order[temp]
    q_i[key]= query_item_order.tolist()

In [ ]:
# Write to a file 
f= open("D:\\Courses\\InfoStorage\\Project\\Data\\submissions\\RankSVMAllFeaturesWithAllEmbeddingsBasic.txt","w")
for key in q_i:
    f.write("%s " % key)
    for index,item in enumerate(q_i.get(key)):
        ll=len(q_i.get(key))-1
        if index < ll:
            f.write("%s," % item)
        else:
            f.write("%s" % item)
    f.write("\n")
f.close()